<a href="https://colab.research.google.com/github/badonyi/adventofcode2024/blob/main/aov2024_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Run this chunk before any of the daily challenges

In [ ]:
options(scipen = 999)
get_data <- function(x) sprintf('https://raw.githubusercontent.com/badonyi/adventofcode2024/refs/heads/main/data/%s.txt', x)

## Day 1

In [ ]:
input <- read.table(get_data('day_1'))

# part 1
sum(abs(sort(input$V1) - sort(input$V2)))

# part 2
sum(outer(input$V1, input$V2, `==`) * input$V1)

## Day 2

In [ ]:
input <- sapply(strsplit(readLines(get_data('day_2')), ' '), as.integer)

# part 1
sum(sapply(input, function(x) {
  d <- diff(x); all(abs(d) < 4) && (all(d < 0) || all(d > 0))
}))

# part 2
sum(sapply(input, function(x) {
  any(sapply(seq_along(x), function(i) {
    d <- diff(x[-i]); all(abs(d) < 4) && (all(d < 0) || all(d > 0))
  }))
}))


## Day 3

In [ ]:
input <- paste(readLines(get_data('day_3')), collapse = '')
pat <- 'mul\\((\\d+),(\\d+)\\)'
mul_compute <- function(s) {
  eval(parse(text = paste(
    gsub(pat, '\\1*\\2', regmatches(s, gregexpr(pat, s))[[1]]), collapse = '+'
  )))
}

# part 1
mul_compute(input)

# part 2
donts <- strsplit(input, "don't\\(\\)")[[1]]
dos <- unlist(lapply(donts[-1], function(x) strsplit(x, 'do\\(\\)')[[1]][-1]))
mul_compute(paste(c(donts[1], dos), collapse = ''))

## Day 4

In [ ]:
mat <- do.call(rbind, strsplit(readLines(get_data('day_4')), NULL))
has_xmas <- function(x) paste(x, collapse = '') %in% c('XMAS', 'SAMX')

# part 1
n <- nrow(mat)
count <- 0
for (j in seq(n)) {
  for (i in seq(n)) {
    if (i <= n - 3) {
      count <- count + has_xmas(mat[j, i:(i + 3)])
      if (j < n - 2) count <- count + has_xmas(diag(mat[j:(j + 3), i:(i + 3)]))
      if (j > 3) count <- count + has_xmas(diag(mat[j:(j - 3), i:(i + 3)]))
    }
    if (j < n - 2) count <- count + has_xmas(mat[j:(j + 3), i])
  }
}
cat(count)

# part 2
sum(apply(
  X = expand.grid(seq(n), seq(n)),
  MARGIN = 1,
  FUN = function(g) {
    a <- g[1]
    b <- g[2]
    if (a > 1 && a < n && b > 1 && b <  n) {
      return(as.numeric(
        paste0(mat[a - 1, b - 1], mat[a - 1, b + 1],
               mat[a, b], mat[a + 1, b - 1],
               mat[a + 1, b + 1]) %in% c('SMASM', 'MSAMS', 'MMASS', 'SSAMM')))
    }
    return(0)
  }
))

## Day 5

In [ ]:
input <- readLines(get_data('day_5'))
sep <- which(input == '')
updates <- strsplit(input[(sep + 1):length(input)], ',')
pairs <- read.table(text = input[1:(sep - 1)], sep = '|')

is_correct <- function(update, pairs) {
  for (i in 1:nrow(pairs)) {
    if (pairs$V1[i] %in% update && pairs$V2[i] %in% update) {
      if (which(update == pairs$V1[i]) > which(update == pairs$V2[i])) {
        return(FALSE)
      }
    }
  }
  return(TRUE)
}

# part 1
sum(sapply(updates[sapply(updates, is_correct, pairs)],
           function(x) as.numeric(x[ceiling(length(x) / 2)])))

# part 2
correct_order <- function(update, pairs) {
  swap <- TRUE
  while (swap) {
    swap <- FALSE
    for (i in 1:nrow(pairs)) {
      if (pairs$V1[i] %in% update && pairs$V2[i] %in% update) {
        b_index <- which(update == pairs$V1[i])
        a_index <- which(update == pairs$V2[i])
        if (b_index > a_index) {
          update[c(a_index, b_index)] <- update[c(b_index, a_index)]
          swap <- TRUE
        }
      }
    }
  }
  return(update)
}

updates_wrong <- updates[!sapply(updates, is_correct, pairs)]
updates_right <- lapply(updates_wrong, correct_order, pairs)
sum(sapply(updates_right, function(x) as.numeric(x[ceiling(length(x) / 2)])))

## Day 6

In [ ]:
input <- do.call(rbind, strsplit(readLines(get_data('day_6')), NULL))
start <- which(input == '^', arr.ind = TRUE)
dirs <- list(c(-1, 0), c(0, 1), c(1, 0), c(0, -1))
cache <- new.env(hash = TRUE, parent = emptyenv())

explore_grid <- function(grid, cur_pos, obstacle = NULL) {
  state <- if (!is.null(obstacle)) {
     paste(obstacle[1], obstacle[2], sep = ',')
  } else {
    'init'
  }

  if (exists(state, envir = cache)) return(get(state, envir = cache))
  if (!is.null(obstacle)) grid[obstacle[1], obstacle[2]] <- '#'

  visited <- matrix(0, nrow = nrow(grid), ncol = ncol(grid))
  cur_dir <- 1
  visited[cur_pos[1], cur_pos[2]] <- cur_dir

  while (TRUE) {
    new_pos <- cur_pos + dirs[[cur_dir]]
    if (new_pos[1] < 1 || new_pos[1] > nrow(grid) ||
        new_pos[2] < 1 || new_pos[2] > ncol(grid)) {
      result <- if (is.null(obstacle)) visited else FALSE
      assign(state, result, envir = cache)
      return(result)
    }

    if (grid[new_pos[1], new_pos[2]] == '#') {
      cur_dir <- (cur_dir %% 4) + 1
    } else {
      if (visited[new_pos[1], new_pos[2]] == cur_dir) {
        result <- if (is.null(obstacle)) visited else TRUE
        assign(state, result, envir = cache)
        return(result)
      }
      visited[new_pos[1], new_pos[2]] <- cur_dir
      cur_pos <- new_pos
    }
  }
}

# part 1
sum(explore_grid(input, start) != 0)

# part 2
all_loc <- which(input == '.', arr.ind = TRUE)
loop_count <- 0
pb <- txtProgressBar(min = 0, max = nrow(all_loc), style = 3)
for (i in seq_len(nrow(all_loc))) {
  loop_count <- loop_count + explore_grid(input, start, all_loc[i, ])
  setTxtProgressBar(pb, i)
}
close(pb)
loop_count

## Day 7

In [ ]:
permutations <- function(n, r, v, repeats.allowed = FALSE) {
  if (r == 0) return(matrix(nrow = 1, ncol = 0))
  if (r == 1) return(matrix(v, ncol = 1))

  result <- NULL
  for (i in seq_along(v)) {
    if (repeats.allowed || !(v[i] %in% v[1:(i - 1)])) {
      sub_perms <- permutations(n, r - 1, v, repeats.allowed)
      perms <- cbind(matrix(v[i], nrow = nrow(sub_perms), ncol = 1), sub_perms)
      result <- rbind(result, perms)
    }
  }
  return(result)
}

run_ops <- function(test_val, num_val, ops) {
  eval_ops <- function(current, remaining, test_val) {
    if (length(remaining) == 0) return(current == test_val)

    for (op in ops) {
      next_num <- remaining[1]
      new_current <- switch(op,
                            '+' = current + next_num,
                            '*' = current * next_num,
                            '||' = as.numeric(paste0(current, next_num)))
      if (eval_ops(new_current, remaining[-1], test_val)) return(TRUE)
    }
    return(FALSE)
  }

  if (length(num_val) == 1) return(ifelse(test_val == num_val[1], test_val, 0))
  if (eval_ops(num_val[1], num_val[-1], test_val)) return(test_val)
  return(0)
}

input <- strsplit(readLines(get_data('day_7')), ': ')
test_val <- as.numeric(sapply(input, `[`, 1))
num_val <- lapply(sapply(input, `[`, 2), function(x) {
  as.numeric(strsplit(x, ' ')[[1]])
})

# part 1
sum(sapply(seq_along(test_val), function(i) {
  run_ops(test_val[i], num_val[[i]], c('+', '*'))
}))

# part 2
sum(sapply(seq_along(test_val), function(i) {
  run_ops(test_val[i], num_val[[i]], c('+', '*', '||'))
}))

## Day 8

In [ ]:
input <- readLines(get_data('day_8'))
mat <- do.call(rbind, strsplit(input, ''))
antennae <- unique(mat[mat != '.'])

get_antinode <- function(mat, freq, extend = FALSE) {
  pos <- which(mat == freq, arr.ind = TRUE)
  if (nrow(pos) < 2) return(matrix(NA, nrow = 0, ncol = 2))
  combs <- combn(seq_len(nrow(pos)), 2)

  results <- NULL
  for (i in seq_len(ncol(combs))) {
    pos1 <- pos[combs[1, i], ]
    pos2 <- pos[combs[2, i], ]
    dist <- pos1 - pos2

    if (extend) {
      for (k in seq(0, ncol(mat))) {
        results <- rbind(results, pos1 + k * dist, pos2 - k * dist)
      }
    } else {
      results <- rbind(results, pos1 + dist, pos2 - dist)
    }
  }

  unique(results[results[, 1] > 0 & results[, 1] <= nrow(mat) &
                 results[, 2] > 0 & results[, 2] <= ncol(mat), ])
}

# part 1
nrow(unique(do.call(
  rbind, lapply(antennae, function(freq)
    get_antinode(mat, freq, extend = FALSE))
)))

# part 2
nrow(unique(do.call(
  rbind, lapply(antennae, function(freq)
    get_antinode(mat, freq, extend = TRUE))
)))

## Day 9

In [ ]:
input <- readLines(get_data('day_9'))

# part 1
digits <- as.numeric(strsplit(input, '')[[1]])
ids <- (seq(digits) - 1) / 2
disk_long <- ids[rep(seq(digits), digits)]
files_to_move <- rev(disk_long[disk_long == floor(disk_long)])
empty_idx <- which(disk_long != floor(disk_long))
disk_long[empty_idx] <- files_to_move[seq(empty_idx)]
sum(disk_long[seq(files_to_move)] * (seq(files_to_move) - 1))

# part 2
blocks <- mapply(rep, times = digits, x = ifelse((!seq(digits) %% 2), -1, ids))
free_space <- digits * (!seq(digits) %% 2)

for (file_id in seq(length(blocks), 2, by = -1)) {
  file <- blocks[[file_id]][blocks[[file_id]] != -1]

  if (length(file) > 0) {
    blocks[[file_id]][] <- -1
    free_space[file_id] <- free_space[file_id] + length(file)
    block <- which(free_space >= length(file))[1]

    if (!is.na(block)) {
      blocks[[block]][blocks[[block]] == -1][seq_along(file)] <- file
      free_space[block] <- free_space[block] - length(file)
    }
  }
}

compact_blocks <- unlist(blocks[digits != 0])
sum((compact_blocks * (compact_blocks > 0)) * (seq(compact_blocks) - 1))

## Day 10

In [ ]:
input <- readLines(get_data('day_10'))
topo_map <- do.call(rbind, lapply(strsplit(input, ''), as.integer))
height <- nrow(topo_map)
width <- ncol(topo_map)
trailheads <- which(topo_map == 0, arr.ind = TRUE)
moves <- list(up = c(-1, 0), down = c(1, 0), left = c(0, -1), right = c(0, 1))

explore_paths <- function(row, col, elevation, visited, end, path, paths) {
  if (row <= 0 || row > height || col <= 0 || col > width ||
      topo_map[row, col] != elevation || visited[row, col]) {
    return(list(end = end, paths = paths))
  }

  path <- rbind(path, c(row, col))

  if (elevation == 9) {
    end <- unique(rbind(end, c(row, col)))
    paths <- c(paths, list(path))
    return(list(end = end, paths = paths))
  }

  visited[row, col] <- TRUE

  result <- lapply(moves, function(move) {
    explore_paths(row + move[1], col + move[2],
                  elevation + 1, visited, end, path, paths)
  })

  end <- do.call(rbind, lapply(result, `[[`, 'end'))
  paths <- do.call(c, lapply(result, `[[`, 'paths'))

  visited[row, col] <- FALSE

  return(list(end = unique(end), paths = paths))
}

visited <- matrix(FALSE, nrow = height, ncol = width)
results <- apply(trailheads, 1, function(start) {
  explore_paths(start[1], start[2], 0, visited,
                end = matrix(nrow = 0, ncol = 2),
                path = matrix(nrow = 0, ncol = 2),
                paths = vector(mode = 'list'))
})

# part 1
sum(sapply(results, function(res) nrow(res$end)))

# part 2
all_paths <- do.call(c, lapply(results, `[[`, 'paths'))
length(unique(sapply(all_paths, function(path) {
  paste(apply(path, 1, paste, collapse = ','), collapse = ';')
})))

## Day 11

In [ ]:
transform_stone <- function(stone) {
  chr <- as.character(stone)
  if (stone == 0) {
    return(1)
  } else if (nchar(chr) %% 2 == 0) {
    return(
      c(
        as.numeric(substr(chr, 1, nchar(chr) / 2)),
        as.numeric(substr(chr, nchar(chr) / 2 + 1, nchar(chr)))
      )
    )
  } else {
    return(stone * 2024)
  }
}

blink <- function(stones, times) {
  tbl <- table(stones)

  for (i in seq(times)) {
    tmp <- unlist(lapply(names(tbl), function(idx) {
      count <- tbl[idx]
      new_stones <- transform_stone(as.numeric(idx))
      rep(count, length(new_stones))
    }), use.names = FALSE)

    names(tmp) <- unlist(lapply(names(tbl), function(idx) {
      new_stones <- transform_stone(as.numeric(idx))
      as.character(new_stones)
    }))

    tbl <- tapply(tmp, names(tmp), sum)
  }
  tbl
}

# part 1
stones <- scan(get_data('day_11'), numeric())
sum(blink(stones, 25))

# part 2
sum(blink(stones, 75))

## Day 12

In [ ]:
input <- strsplit(readLines(get_data('day_12')), '')
grid <- do.call(rbind, input)

count_sides <- function(seen) {
  pad <- rbind(FALSE, cbind(FALSE, seen, FALSE), FALSE)
  count_sides <- function(seen) {
    sides <- 0
    for (i in seq(nrow(seen) - 1)) {
      row1 <- seen[i, ]
      row2 <- seen[i + 1, ]
      sides <- sides + sum(diff(c(FALSE, row1 & !row2)) == 1) +
        sum(diff(c(FALSE, row2 & !row1)) == 1)
    }
    sides
  }
  return(count_sides(pad) + count_sides(t(pad)))
}

block_price <- function(grid, coord, sides = FALSE) {
  dir <- list(c(-1, 0), c(1, 0), c(0, -1), c(0, 1))
  seen <- matrix(FALSE, nrow = nrow(grid), ncol = ncol(grid))
  block <- grid[coord[1], coord[2]]
  coords <- list(coord)
  area <- perim <- 0

  while (length(coords) > 0) {
    idx <- matrix(coords[[1]], nrow = 1)
    coords <- coords[-1]

    if (any(idx <= 0) | any(idx > dim(grid))) {
      perim <- perim + 1
      next
    }

    if (grid[idx] != block) {
      perim <- perim + 1
      next
    }

    if (seen[idx]) {
      next
    } else {
      seen[idx] <- TRUE
    }

    area <- area + 1
    coords <- c(coords, lapply(dir, function(x) idx + x))
  }

  if (sides) {
    perim <- count_sides(seen)
  }

  grid[seen] <- 'seen'
  return(list(grid = grid, price = perim * area))
}

calculate_price <- function(grid, sides = FALSE) {
  price <- 0
  while (length(block_coord <- which(grid != 'seen', arr.ind = TRUE)) > 0) {
    coord <- block_coord[1, , drop = FALSE]
    block <- block_price(grid, coord, sides)
    grid <- block$grid
    price <- price + block$price
  }
  return(price)
}

# part 1
calculate_price(grid, sides = FALSE)

# part 2
calculate_price(grid, sides = TRUE)

## Day 13

In [ ]:
input <- lapply(
  strsplit(paste0(readLines(get_data('day_13')), collapse = '\n'), '\n\n')[[1]],
  function(x) as.numeric(unlist(regmatches(x, gregexpr('[+\\-]?\\d+', x))))
)

start_claw <- function(part2 = FALSE) {
  sum(sapply(input, function(i) {
    x <- matrix(i[1:4], 2)
    y <- matrix(i[5:6], 2) + if (part2) 1e13 else 0
    z <- solve(x, y)
    if (all(x %*% round(z) == y) && all(z > 0) && (all(z <= 1e2) || part2))
      sum(z * c(3, 1)) else 0
  }))
}

# part 1
start_claw()

# part 2
start_claw(part2 = TRUE)

## Day 14

In [ ]:
input <- readLines(get_data('day_14'))
input <- lapply(regmatches(input, gregexpr('-?\\d+', input)), as.numeric)

safety_val <- function(t) {
  final_positions <- function(t) {
    do.call(rbind, lapply(input, function(r) {
      px <- r[1]; py <- r[2]; vx <- r[3]; vy <- r[4]
      c((px + vx * t) %% 101, (py + vy * t) %% 103)
    }))
  }

  prod(table(apply(final_positions(t), 1, function(xy) {
    x <- xy[1]; y <- xy[2]; mx <- 101 %/% 2; my <- 103 %/% 2
    if (x < mx && y < my) return(1)
    if (x > mx && y < my) return(2)
    if (x < mx && y > my) return(3)
    if (x > mx && y > my) return(4)
    return(NA)
  })))
}

# part 1
safety_val(100)

# part 2
which.min(sapply(seq.int(1, 1e4), function(t) safety_val(t)))